In [1]:
import torch as tc
import numpy as np
import pandas as pd

from star.star import STAR
from curve_utils import CurveUtils
from curve_generator import CurveGenerator, get_all_curves, select_better
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

In [2]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Inside leg height',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'stature', # 5.1.1
    'crotch_height', # 5.1.15
    'shoulder_width', # 5.4.3
]
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
}

# Prepating Bodies

In [3]:
for gender in ['female', 'male']:
    betas = tc.FloatTensor(np.load(f'data/{gender}_betas.npy')).to(device)
    trans = tc.FloatTensor(np.load(f'data/{gender}_trans.npy')).to(device)
    star = STAR(gender=gender, passed_path=f"star/{gender}.npz", num_betas=betas.shape[-1])
    
    poses = tc.FloatTensor(np.load(f'data/{gender}_poses.npy')).to(device)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_a.pt')
    
    poses = np.zeros((betas.shape[0],72))
    poses[:,5] = 6 * np.pi/180
    poses[:,8] = -6 * np.pi/180
    poses = tc.cuda.FloatTensor(poses)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_t.pt')

template = star.faces.to(device)

In [4]:
%%script false --no-raise-error
for gender in ['female', 'male']:
    bodies = tc.load(f'data/{gender}_bodies_a.pt')
    for idx, body in enumerate(bodies):
            CurveUtils.save_obj(f"tests/{gender}_{idx}_body.obj", body, star.faces+1)

In [5]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    measures.index = measures['Subject']
    measures = measures[measures['Measuring station'] == "MOVE4D"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures._get_numeric_data().mean(axis=0)
    # selected_subjects[gender] = abs(temp_measures - mean_measures).sum(axis=1).idxmin()
    selected_subjects[gender] = 'IEEEP2_01' if gender == 'female' else 'IEEEP2_02'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [6]:
all_data = []
all_best_curves = []
for gender in ['female', 'male']:
    print(f'MEASURING {gender.upper()} BODY')
    bodies = tc.load(f'data/{gender}_bodies_t.pt')
    body = bodies[selected_subjects[gender]]
    measures = selected_measures[gender]
    result = get_all_curves(body, measures, template, device)
    best_curves, data = select_better(result, body, measures, device)
    tc.save(best_curves, f'data/{gender}_best_curves.zip')

MEASURING FEMALE BODY BASE
MEASURING MALE BODY BASE


In [7]:
all_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    all_measures[gender] = dict()
    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        temp_measures = measures[measures['Measuring station'] == station]
        temp_measures = temp_measures[temp_measures['Phase'] == 2]
        temp_measures = temp_measures[their_semantics]
        temp_measures.columns = our_semantic
        all_measures[gender][station] = temp_measures

In [9]:
tc.set_printoptions(precision=2, sci_mode=False, linewidth=100)

for gender in ['female', 'male']:
    bodies = tc.load(f'data/{gender}_bodies_a.pt')
    best_curves = tc.load(f'data/{gender}_best_curves.zip')

    all_bodies_measures = []
    for idx,body in enumerate(bodies):
        all_curves_measures = []
        all_positions = []
        for curve in best_curves:
            position = CurveGenerator.generate_positions(body, curve)[::2]
            # position, measures = CurveUtils.ray_polygon_cosilion(body,template,position)
            all_positions.extend(position.cpu().numpy())
            measures = CurveGenerator.calculate_distances(position)
            all_curves_measures.append(measures)
        all_bodies_measures.append(all_curves_measures)
        CurveUtils.save_obj(f"tests/{gender}_{idx}_curves.obj", all_positions)
    all_bodies_measures = tc.FloatTensor(all_bodies_measures)
    our_measures = all_bodies_measures

    tempdata = pd.read_pickle(f'data/{gender}_measures.zip')
    tempdata = tempdata[tempdata['Measuring station'] == 'MOVE4D']
    tempdata = tempdata['Age (years)']
    data_to_save = pd.DataFrame(our_measures, index=tempdata.index,columns=curve_index.keys())
    data_to_save['age'] = tempdata
    data_to_save.to_csv(f'data/{gender}_calculated_measures.csv')

    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        print(f"\n\nmeasures difference from {station}:")
        
        ieee_measures = tc.FloatTensor(all_measures[gender][station][list(curve_index.keys())].to_numpy())
        diff = abs(ieee_measures - (our_measures*10))

        print(pd.DataFrame({
            "mean": diff.mean(axis=0),
            "std": diff.std(axis=0),
            "min": diff.min(axis=0).values,
            "max": diff.max(axis=0).values,
        }, index=list(curve_index.keys())))



measures difference from MOVE4D:
                       mean        std       min        max
neck_girth         7.213131   5.445031  0.029602  21.260345
bust_chest_girth  14.615535  10.640256  0.168945  39.519714
waist_girth       10.799312  10.559777  0.033813  44.973267
hip_girth         15.180845  14.894855  0.023376  62.328125
upper_arm_girth    3.908664   3.441707  0.073303  12.717987
thigh_girth       11.757521   6.062829  1.384216  25.540527


measures difference from Expert measurer 1:
                       mean        std       min        max
neck_girth         9.933315   8.248340  0.115265  31.211517
bust_chest_girth  18.382940  13.643857  0.030579  66.691650
waist_girth       28.263540  17.901636  0.182739  71.637939
hip_girth         15.499597  11.753053  0.265137  55.058105
upper_arm_girth   10.953380  10.588461  0.477203  75.060242
thigh_girth       12.212731   8.716128  0.095886  43.339905


measures difference from Expert measurer 5:
                       mean      